In [3]:
from lxml import html
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import time
import json

path_to_driver = '/Users/rogotulka/Downloads/chromedriver'
path_to_app = 'https://play.google.com/store/apps/details?id=ru.sberbankmobile'
delay = 5

browser = webdriver.Chrome(executable_path=path_to_driver)
browser.get(path_to_app)

# собираем отзывы, даты и авторов с оценками
reviews = []
raits = []
auths = []
dates = []
batch_reviews = []
batch_raits = []
batch_auths= []
batch_dates = []
try:
    # нужно сделать пару переходов по кнопкам, чтобы начать собирать отзывы    
    elem = browser.find_element_by_xpath(
            '//*[@id="body-content"]/div/div/div[1]/div[2]/div[2]/div[1]/div[4]/button[2]/div[2]/div/div') 
    elem.click()
    time.sleep(4)
    elem = browser.find_element_by_xpath(
            '//*[@id="body-content"]/div/div/div[1]/div[2]/div[2]/div[1]/div[3]/div[1]/button') 
    elem.click()
    time.sleep(4)
    elem = browser.find_element_by_xpath(
            '//*[@id="body-content"]/div/div/div[1]/div[2]/div[2]/div[1]/div[3]/div[1]/div/ul/li[3]/button')
    time.sleep(2)
    elem.click()
except Exception:
    time.sleep(5)
    

for i in range(1000):
    batch_reviews = []
    batch_raits = []
    batch_auths= []
    batch_dates = []
    for j in range(20):   
        batch_reviews.extend(reviews)
        batch_raits.extend(raits)
        batch_auths.extend(auths)
        batch_dates.extend(dates)
        reviews = []
        raits = []
        auths = []
        dates = []
        #  плохой интернет - ждем загрузки кнопки
        try:
            myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, 
                                '//*[@id="body-content"]/div/div/div[1]/div[2]/div[2]/div[1]/div[4]/button[2]/div[2]/div/div')))
        except TimeoutException:
            print("Loading took too much time!")

        #  нажимаем стрелку далее
        try:
            elem = browser.find_element_by_xpath(
                '//*[@id="body-content"]/div/div/div[1]/div[2]/div[2]/div[1]/div[4]/button[2]/div[2]/div/div') 
            elem.click()
        except Exception:
            time.sleep(5)
        
        tree = html.fromstring(browser.page_source)
        elem = tree.xpath('//div[@class="multicol-column" and contains(@style, "visibility: visible")]//div[@class="review-body with-review-wrapper"]//text()')
        auths = tree.xpath('//div[@class="multicol-column" and contains(@style, "visibility: visible")]//span[@class="author-name"]//text()')
        dates = tree.xpath('//div[@class="multicol-column" and contains(@style, "visibility: visible")]//span[@class="review-date"]//text()')
        reviews = []
        # собираем отзывы, выкидывая лишнее
        for el in elem:
            if len(el.strip()) != 0 and el.strip()!='Читать дальше':
                reviews.append(str(el))
        # собираем рейтинги
        rat = tree.xpath('//div[@class="multicol-column" and contains(@style, "visibility: visible")]//div[@class="review-info-star-rating"]//@aria-label')
        # отрезаем оценку из текста        
        for el in rat:
            raits.append(el[-6])
    #  сохраним batch отзывов
    with open('data/reviews'+str(i)+'.txt', 'w') as file:
        file.write('\n'.join('{}\t{}\t{}\t{}'.format(x[0],x[1], x[2], x[3]) for x in list(zip(batch_reviews, batch_auths, batch_dates, batch_raits))))

    # из-за того, что гугл дозаписывает все новые отзывы в dom элементы - я их удаляю, иначе браузер 
    element = tree.xpath('//div[@class="expand-page"]')
    for l in range(len(element)):
        browser.execute_script("document.getElementsByClassName('expand-page')[0].remove();")
    tree = html.fromstring(browser.page_source)
    element = tree.xpath('//div[@class="expand-page"]')

browser.quit()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=59.0.3071.115)
  (Driver info: chromedriver=2.30.477690 (c53f4ad87510ee97b5c3425a14c0e79780cdf262),platform=Mac OS X 10.11.6 x86_64)
